# Creating geometrical phantoms to test the effect of bone and trachea geomtry in the trend of values

In [30]:
import numpy as np
import nibabel as nib

def create_test_shape(volume_size, cylinder_radius, cylinder_height, cube_size, cube_spacing, rectangle_size, rectangle_spacing):
    # Initialize 3D volume
    volume = np.zeros(volume_size)
    volume[:, :, :] = -9.05
    # Center position for the cylinder
    center = np.array(volume_size) // 2

    # Cilinder == canal in the center
    for z in range(int(center[2] - cylinder_height // 2), int(center[2] + cylinder_height // 2)):
        for x in range(volume_size[0]):
            for y in range(volume_size[1]):
                if (x - center[0]) ** 2 + (y - center[1]) ** 2 < cylinder_radius ** 2:
                    volume[x, y, z] = -9.055  # Chi value of canal

    # Create the squares - bones spine in posterior 
    for i in range(3):
        cube_x_start = int(center[0] - cylinder_radius - cube_size - 10)  # Offset from cylinder
        cube_y_start = int(center[1] - cube_size // 2)  # Centered vertically
        cube_z_start = int(center[2] - (1.5 * cube_size) + i * (cube_size + cube_spacing))
        volume[cube_x_start:cube_x_start + cube_size,
               cube_y_start:cube_y_start + cube_size,
               cube_z_start:cube_z_start + cube_size] = -11  # Set to unique value for red cubes

    # Create the anterior rectangles
    for i in range(3):
        rect_x_start = int(center[0] + cylinder_radius + 10)  # Offset from cylinder
        rect_y_start = int(center[1] - rectangle_size[1] // 2)  # Centered vertically
        rect_z_start = int(center[2] - (1.5 * rectangle_size[0]) + i * (rectangle_size[0] + rectangle_spacing))
        volume[rect_x_start:rect_x_start + rectangle_size[0],
               rect_y_start:rect_y_start + rectangle_size[1],
               rect_z_start:rect_z_start + rectangle_size[0]] = -11  # Set to unique value for purple rectangles

    # Save as NIfTI file
    affine = np.eye(4)  # Identity affine matrix for simplicity
    nifti_img = nib.Nifti1Image(volume, affine)
    nib.save(nifti_img, 'custom_test_shape_aligned3.nii.gz')
    print("Saved 3D shape as 'custom_test_shape_aligned3.nii.gz'")



In [31]:
# Parameters
volume_size = (100, 100, 100)       # Volume size (modify as needed)
cylinder_radius = 10                # Radius of the central cylinder
cylinder_height = 100                # Height of the central cylinder
cube_size = 10                      # Side length of red cubes
cube_spacing = 5                   # Spacing of the anterior bones (smaller bones)
rectangle_size = (20, 10)           # Dimensions of purple rectangles (length, width)
rectangle_spacing = 15             # Spacing of the posterior bones (big bones)

# Run the function
create_test_shape(volume_size, cylinder_radius, cylinder_height, cube_size, cube_spacing, rectangle_size, rectangle_spacing)

Saved 3D shape as 'custom_test_shape_aligned3.nii.gz'


In [18]:
# Now the mask
import numpy as np
import nibabel as nib

# Path to the existing NIfTI file with the cylinder
nifti_path = 'custom_test_shape_aligned.nii.gz'

# Load the NIfTI image
img = nib.load(nifti_path)
data = img.get_fdata()

# Get the affine matrix for saving the new NIfTI
affine = img.affine


In [19]:
# Define the value assigned to the cylinder in the original volume
cylinder_value = -9.055

# Create a mask for the cylinder
cylinder_mask = (data == cylinder_value)


In [24]:
# Get the dimensions of the volume
dim_x, dim_y, dim_z = data.shape

# Calculate the center of the volume
center_x, center_y, center_z = dim_x // 2, dim_y // 2, dim_z // 2

# Initialize a new volume for the segmented cylinder
segmented_cylinder = np.zeros_like(data)

# Define the four quadrants
# Quadrant I: x >= center_x and y >= center_y
# Quadrant II: x < center_x and y >= center_y
# Quadrant III: x < center_x and y < center_y
# Quadrant IV: x >= center_x and y < center_y

# Quadrant I
segmented_cylinder[np.logical_and(cylinder_mask,
                                  np.logical_and(np.arange(dim_x)[:, None, None] >= center_z,
                                                 np.arange(dim_y)[None, :, None] >= center_x))] = 1

# Quadrant II
segmented_cylinder[np.logical_and(cylinder_mask,
                                  np.logical_and(np.arange(dim_x)[:, None, None] < center_z,
                                                 np.arange(dim_y)[None, :, None] >= center_x))] = 2

# Quadrant III
segmented_cylinder[np.logical_and(cylinder_mask,
                                  np.logical_and(np.arange(dim_x)[:, None, None] < center_z,
                                                 np.arange(dim_y)[None, :, None] < center_x))] = 3

# Quadrant IV
segmented_cylinder[np.logical_and(cylinder_mask,
                                  np.logical_and(np.arange(dim_x)[:, None, None] >= center_z,
                                                 np.arange(dim_y)[None, :, None] < center_x))] = 4


In [25]:
# Save the segmented cylinder as a new NIfTI file
segmented_nifti = nib.Nifti1Image(segmented_cylinder, affine)
nib.save(segmented_nifti, 'segmented_cylinder_four_sections.nii.gz')

print("Saved segmented cylinder with four sections as 'segmented_cylinder_four_sections.nii.gz'")


Saved segmented cylinder with four sections as 'segmented_cylinder_four_sections.nii.gz'
